In [ ]:
# Notebook vorbereiten
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil 
ab_ag_raw = pd.read_csv('data/ab_ag.tsv', sep='\t')
columns = pd.read_csv('data/columns.tsv', sep='\t')
uniprot_data = pd.read_csv('data/uniprot_data.tsv', sep='\t')

# ab_ag aus ab_ag_raw aufbereiten
ab_ag = ab_ag_raw[["pdb", "Hchain", "Lchain", "model", "antigen_chain", "antigen_name", "antigen_species"]]
# Füge Spalten für die CDR-Sequenzen in ab_ag hinzu
for col in ["CDR_H1", "CDR_H2", "CDR_H3", "CDR_L1", "CDR_L2", "CDR_L3"]:
    if col not in ab_ag.columns:
        ab_ag[col] = np.nan
    ab_ag[col] = ab_ag[col].astype(object)
# Entferne Einträge ohne Antigennamen
ab_ag.dropna(subset=["antigen_chain"], inplace=True)

# Zeilen entfernen, bei denen resolution <= 3.5
# ACHTUNG! Das überschreibt die vorherige Variable ab_ag! Warum kommt der Code unten nochmal?
ab_ag = pd.read_csv("data/ab_ag.tsv", sep="\t") 
ab_ag_resolution = ab_ag[ab_ag['resolution'] <= 3.5]

# Ich hab aus Versehen das Code-Feld gelöscht! Code von Anna für doppelte pdb Einträge zurückholen!

C:\Users\marce\AppData\Local\Temp\ipykernel_4256\2058240722.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ab_ag[col] = np.nan
C:\Users\marce\AppData\Local\Temp\ipykernel_4256\2058240722.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ab_ag[col] = ab_ag[col].astype(object)
C:\Users\marce\AppData\Local\Temp\ipykernel_4256\2058240722.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [3]:
#### Überarbeiteter Code zum Herunterladen der Antikörpersequenzen. Nach dem Download wird der Code nicht mehr benötigt, da die Daten in ab_ag.tsv gespeichert werden.
## Funktionen aus dem sabdab_downloader importieren
#from common.sabdab_downloader import getannotation
#
## Definiere CDR-Regionen als slices für die H- und L-Ketten (Literatur überprüfen, ob die Indizes korrekt sind!)
#cdr_regions = {
#        "H1": slice(26, 32),
#        "H2": slice(52, 56),
#        "H3": slice(95, 102),
#        "L1": slice(24, 34),
#        "L2": slice(50, 56),
#        "L3": slice(89, 97)
#}
#
### Lade die Chothia-Daten und Chothia-nummerierten Sequenzen für jeden PDB-Eintrag herunter und speichere sie im Ordner des entsprechenden PDB-Eintrags:
## Für jede Zeile (bzw. Index, da einige Zeilen aus ab_ag_raw gelöscht wurden) in ab_ag
#for x in ab_ag.index:
#    # Extrahiere die PDB-ID, H-Kette, L-Kette und Modellnummer aus der Zeile für die Argumente von getannotation:
#    fab_unique = ab_ag.loc[x,["pdb", "Hchain", "Lchain", "model"]].tolist()
#    pdb = fab_unique[0] # PDB-ID aus der Liste extrahieren
#    # Erstelle einen temporären Ordner in sabdab_download, falls er noch nicht existiert für die files, die getannotation herunterladen wird
#    if not os.path.exists(f"data/sabdab_download/{pdb}"):
#        os.makedirs(f"data/sabdab_download/{pdb}")
#    # Lade die Chothia-nummerierten Sequenzen herunter oder gib eine Fehlermeldung für entsprechende PDB-Einträge aus.
#    # getanntation erwartet als zweites Argument eine Liste aus einer oder mehreren Listen. Wir benutzen pro Zeile in ab_ag jeweils eine Liste, die eine einzige Liste enthält:
#    try:
#        getannotation(pdb, [fab_unique], f"data/sabdab_download/{pdb}")
#        # Speichere die heruntergeladenen .ann-Dateien für VH und VL jeweils in einem DataFrame:
#        VH_chothia_seq = pd.read_csv(f"data/sabdab_download/{pdb}/{pdb}_{fab_unique[1]}_VH.ann", sep='\t', header=None, names=['Chothia_Position', 'Aminosäure'])
#        VL_chothia_seq = pd.read_csv(f"data/sabdab_download/{pdb}/{pdb}_{fab_unique[2]}_VL.ann", sep='\t', header=None, names=['Chothia_Position', 'Aminosäure'])
#        # Extrahiere die CDR-Sequenzen aus den Chothia-annotierten Sequenzen und speichere sie in ab_ag
#        H1_seq = VH_chothia_seq.loc[cdr_regions["H1"], "Aminosäure"].tolist()
#        ab_ag.at[x, "CDR_H1"] = H1_seq
#        H2_seq = VH_chothia_seq.loc[cdr_regions["H2"], "Aminosäure"].tolist()
#        ab_ag.at[x, "CDR_H2"] = H2_seq
#        H3_seq = VH_chothia_seq.loc[cdr_regions["H3"], "Aminosäure"].tolist()
#        ab_ag.at[x, "CDR_H3"] = H3_seq
#        L1_seq = VL_chothia_seq.loc[cdr_regions["L1"], "Aminosäure"].tolist()
#        ab_ag.at[x, "CDR_L1"] = L1_seq
#        L2_seq = VL_chothia_seq.loc[cdr_regions["L2"], "Aminosäure"].tolist()
#        ab_ag.at[x, "CDR_L2"] = L2_seq
#        L3_seq = VL_chothia_seq.loc[cdr_regions["L3"], "Aminosäure"].tolist()
#        ab_ag.at[x, "CDR_L3"] = L3_seq
#    except Exception as e:
#        print(f"Could not download Chothia-annotated sequence for {pdb}: {e}")
#    # Lösche den temporären Ordner, nachdem die Sequenzen extrahiert wurden:
#    shutil.rmtree(f"data/sabdab_download/{pdb}")
#ab_ag.to_csv('data/ab_ag_annotated.tsv', sep='\t', index=False)

In [ ]:
#Kopie des annotated datensatzes erstellen
ab_ag_annotated_copy = pd.read_csv('data/ab_ag_annotated.tsv', sep='\t')
#anzahl der einträge vor der Löschung zur Kontrolle
count_beforedeletion = len(ab_ag_annotated_copy)
#löschung der einträge mit identischer sequenz
ab_ag_uniquesequences = ab_ag_annotated_copy.drop_duplicates(subset=['pdb', 'CDR_H1','CDR_H2','CDR_H3', 'CDR_L1', 'CDR_L2', 'CDR_L3'] )
#anzahl der einträge nach der löschung zur Kontrolle
duplicates_removed = count_beforedeletion - len(ab_ag_uniquesequences)
#mit f" können variablen in strings eingebaut werden
print(f"Anzahl gelöschter Einträge beträgt: {duplicates_removed}")

FileNotFoundError: [Errno 2] No such file or directory: '../data/ab_ag_annotated.tsv'

In [ ]:
# Zeilen entfernen, bei denen resolution <= 3.5
import pandas as pd
ab_ag = pd.read_csv("data/ab_ag.tsv", sep="\t") 

ab_ag_resolution = ab_ag[ab_ag['resolution'] <= 3.5]
